In [22]:
import cv2
import numpy as np

In [23]:
def intersects(a, b, min_area_percent=0):
    x1, y1, x2, y2 = a
    x3, y3, x4, y4 = b
    x_overlap = max(0, min(x2, x4) - max(x1, x3))
    y_overlap = max(0, min(y2, y4) - max(y1, y3))
    overlap_area = x_overlap * y_overlap
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x4 - x3) * (y4 - y3)
    return overlap_area > min_area_percent * min(area1, area2)


def merge(a, b):
    x1 = min(a[0], b[0])
    y1 = min(a[1], b[1])
    x2 = max(a[2], b[2])
    y2 = max(a[3], b[3])
    return (x1, y1, x2, y2)


def merge_all(rects):
    hasMerge = True
    while hasMerge:
        hasMerge = False
        i = 0
        while i < len(rects) - 1:
            j = i + 1
            while j < len(rects):
                if intersects(rects[i], rects[j], 0.9):
                    hasMerge = True
                    rects[i] = merge(rects[i], rects[j])
                    del rects[j]
                else:
                    j += 1
            i += 1


def merge_noncontinuous(rects):
    avg_width = sum(x2 - x1 for (x1, y1, x2, y2) in rects) / len(rects)
    avg_height = sum(y2 - y1 for (x1, y1, x2, y2) in rects) / len(rects)
    avg_dst = np.sqrt(avg_width**2 + avg_height**2)
    print("avg dims:", avg_width, avg_height)

    hasMerge = True
    while hasMerge:
        hasMerge = False
        i = 0
        while i < len(rects) - 1:
            x1, y1, x2, y2 = rects[i]
            cx1 = (x1 + x2) / 2
            cy1 = (y1 + y2) / 2
            j = i + 1
            while j < len(rects):
                x3, y3, x4, y4 = rects[j]
                cx2 = (x3 + x4) / 2
                cy2 = (y3 + y4) / 2
                dst = np.sqrt((cx1 - cx2) ** 2 + (cy1 - cy2) ** 2)
                if ((y2 < y3) or (y4 < y1)) and dst < 4.0 * avg_dst:
                    hasMerge = True
                    rects[i] = merge(rects[i], rects[j])
                    del rects[j]
                else:
                    j += 1
            i += 1


def filter_rects(rects, min_area, max_area):
    filtered = []
    for x1, y1, x2, y2 in rects:
        w = x2 - x1
        h = y2 - y1
        a = w * h
        if a < min_area or a > max_area:
            continue
        filtered.append((x1, y1, x2, y2))
    return filtered

In [24]:
symbol_name = "colon"
datasheet_name = "colon"

img = cv2.imread(f"{datasheet_name}.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray, 100, 200)
contours, _ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Bounding boxes
rects = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h
    rects.append((x1, y1, x2, y2))

# Min area relative to max area found
max_a = 0
for x1, y1, x2, y2 in rects:
    a = (x2 - x1) * (y2 - y1)
    max_a = max(max_a, a)
min_area = max_a * 0.2
max_area = 100000

# Merge and clean intersecting rectangles, filter by area
merge_all(rects)
rects = filter_rects(rects, min_area, max_area)

# Non-continuous characters: i, j, :, and =
merge_noncontinuous(rects)

# Draw bounding boxes
for x1, y1, x2, y2 in rects:
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv2.imwrite(f"{datasheet_name}_boxes.jpg", img)

for i, (x1, y1, x2, y2) in enumerate(rects):
    # Get region of interest from grayscale image
    roi = gray[y1:y2, x1:x2]

    # Flip black and white
    flipped = cv2.bitwise_not(roi)

    # Ensure black background
    flipped[flipped < 85] = 0

    # Gaussian blur
    blurred = cv2.GaussianBlur(flipped, ksize=(3, 3), sigmaX=1, sigmaY=1)

    # Resize, maintain aspect ratio, shrink larger dimension to 24 pixels
    resize_size = 24
    width = x2 - x1
    height = y2 - y1
    if width > height:
        new_width = resize_size
        new_height = int(new_width * (height / width))
    else:
        new_height = resize_size
        new_width = int(new_height * (width / height))
    resized = cv2.resize(blurred, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

    # Add 2 pixel black padding
    pad_size = 28
    padded = np.zeros((pad_size, pad_size))
    x_start = pad_size // 2 - new_width // 2
    x_end = x_start + new_width
    y_start = pad_size // 2 - new_height // 2
    y_end = y_start + new_height
    padded[y_start:y_end, x_start:x_end] = resized

    cv2.imwrite(f"data/custom_symbols/{symbol_name}/{symbol_name}_{i}.jpg", padded)

avg dims: 27.9 29.7
